In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
import folium

In [2]:
df = pd.read_excel("x_points_de_collecte.xlsx")
df

,Name,Adresse 1,Adresse 2,Code Postal,Ville,Dép.,Nb Casiers,Heure d'ouverture collecte
0,ELDORA D'OZ,7 rue Laporte,NaN,69009,LYON,RHÔNE,10,10h - 19h
1,PRAIRIAL,10 Rue des Droits de l'Homme,NaN,69120,VAULX-EN-VELIN,RHÔNE,13,10h
2,3 PTITS POIS,124 Rue Sébastien Gryphe,NaN,69007,LYON,RHÔNE,15,NaN
3,BIERISTAN,14 Rue Paul Lafargue,NaN,69100,VILLEURBANNE,RHÔNE,10,du mardi au samedi entre 12h et 16h (A partir ...
4,PASSION BIERES,72 Avenue de Verdun,NaN,69330,MEYZIEU,RHÔNE,12,NaN
...,...,...,...,...,...,...,...,...
145,DAY BY DAY BOURG-EN-BRESSE,37-39 avenue Bad Kreuznach,NaN,1000,BOURG-EN-BRESSE,AIN,12,NaN
146,LA VIE CLAIRE MABLY,"22, rue de l'Artisanat",NaN,42300,MABLY,LOIRE,20,9h - 18h
147,MA PETITE FOURMI,16 RUE ROMARIN,NaN,69001,LYON,RHÔNE,0,NaN
148,BRASSERIE LA CANAILLE,1 Montée Des Curistes,NaN,42890,SAIL-SOUS-COUZAN,LOIRE,0,NaN


In [3]:
geocoder = Nominatim(user_agent="myGeocoder")
adresse_depot = "31 QUAI HIPPOLYTE JAYR, 69009 LYON, RHONE"
location = geocoder.geocode(adresse_depot)
location_depot = (location.latitude, location.longitude)
print(location_depot)
location

(45.7760749, 4.8074721)


Location(31, Quai Hippolyte Jaÿr, Vaise, Lyon 9e Arrondissement, Lyon, Métropole de Lyon, Rhône, Auvergne-Rhône-Alpes, France métropolitaine, 69009, France, (45.7760749, 4.8074721, 0.0))

In [4]:
def get_location(row):
    geocoder = Nominatim(user_agent="myGeocoder")
    adresse = "{name}, {ville}".format(name=row['Name'], ville=row['Ville'])
    location = geocoder.geocode(adresse)
    if location == None:
        #print("Not find with name only " + row['Name'])
        adresse = "{adresse}, {code} {ville}, {dep}".format(adresse=row['Adresse 1'], code=row['Code Postal'], ville=row['Ville'], dep=row['Dép.'])
        location = geocoder.geocode(adresse)
        if location == None:
            print("Wrong adress : " + adresse)
            return None
    return pd.Series([location.latitude, location.longitude])

In [5]:
df[['latitude', 'longitude']] = df.apply(get_location,axis=1)

In [6]:
#Vérification que toutes les adresses ont été trouvées
df.loc[df['latitude'].isna()]

,Name,Adresse 1,Adresse 2,Code Postal,Ville,Dép.,Nb Casiers,Heure d'ouverture collecte,latitude,longitude


In [9]:
#Affichage
map = folium.Map(location=location_depot, zoom_start=9)
df.apply(lambda row:folium.Marker(location=(row['latitude'], row['longitude']), popup=row['Name']).add_to(map), axis=1)
folium.Marker(location=location_depot, popup='Rebooteille', icon=folium.Icon(color="orange")).add_to(map)
map

In [11]:
df.to_excel("data.xlsx", index=False)
df

,Name,Adresse 1,Adresse 2,Code Postal,Ville,Dép.,Nb Casiers,Heure d'ouverture collecte,latitude,longitude
0,ELDORA D'OZ,7 rue Laporte,NaN,69009,LYON,RHÔNE,10,10h - 19h,45.778564,4.804763
1,PRAIRIAL,10 Rue des Droits de l'Homme,NaN,69120,VAULX-EN-VELIN,RHÔNE,13,10h,45.767088,4.927206
2,3 PTITS POIS,124 Rue Sébastien Gryphe,NaN,69007,LYON,RHÔNE,15,NaN,45.748340,4.839300
3,BIERISTAN,14 Rue Paul Lafargue,NaN,69100,VILLEURBANNE,RHÔNE,10,du mardi au samedi entre 12h et 16h (A partir ...,45.767907,4.883542
4,PASSION BIERES,72 Avenue de Verdun,NaN,69330,MEYZIEU,RHÔNE,12,NaN,45.777314,4.992986
...,...,...,...,...,...,...,...,...,...,...
145,DAY BY DAY BOURG-EN-BRESSE,37-39 avenue Bad Kreuznach,NaN,1000,BOURG-EN-BRESSE,AIN,12,NaN,46.222746,5.251099
146,LA VIE CLAIRE MABLY,"22, rue de l'Artisanat",NaN,42300,MABLY,LOIRE,20,9h - 18h,46.068181,4.049424
147,MA PETITE FOURMI,16 RUE ROMARIN,NaN,69001,LYON,RHÔNE,0,NaN,45.768337,4.834245
148,BRASSERIE LA CANAILLE,1 Montée Des Curistes,NaN,42890,SAIL-SOUS-COUZAN,LOIRE,0,NaN,45.734019,3.969524
